<a href="https://colab.research.google.com/github/fxnnxc/Youtube_BigData/blob/master/CrawlingA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Youtube 댓글 Crawling

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=chrome_options)
driver =webdriver.Chrome('chromedriver',options=chrome_options)
#wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
#driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-updates/multiverse amd64 Packages [27.2 kB]
Get:15 http:

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import time

def get_comments_df(url):
    driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)
    driver.get(url)
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(4.0)       # 인터발 1이상으로 줘야 데이터 취득가능(롤링시 데이터 로딩 시간 때문)
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

        if new_page_height == last_page_height:
            break
        last_page_height = new_page_height
    html_source = driver.page_source
    

    # HTML 태크 크롤링 작업
    soup = BeautifulSoup(html_source, "lxml")
    youtube_user_IDs = soup.select("div#header-author > a > span")
    youtube_comments = soup.select("yt-formatted-string#content-text")
    youtube_likes = soup.select("span#vote-count-middle")
    str_youtube_userIDs = []   # USER ID 배열
    str_youtube_comments = []  # USER 댓글 내용 배열
    str_like = []

    # REPLACE DATA
    for i in range(len(youtube_user_IDs)):
        str_tmp = str(youtube_user_IDs[i].text)
        str_tmp = str_tmp.replace('\n', '')
        str_tmp = str_tmp.replace('\t', '')
        str_tmp = str_tmp.replace('   ','')
        str_youtube_userIDs.append(str_tmp)

        str_tmp = str(youtube_comments[i].text)
        str_tmp = str_tmp.replace('\n', '')
        str_tmp = str_tmp.replace('\t', '')
        str_tmp = str_tmp.replace('   ','')
        str_youtube_comments.append(str_tmp)
        
        str_temp = str(youtube_likes[i].text)
        str_temp = str_temp.replace('\n','')
        str_temp = str_temp.replace(" ", '')
        str_like.append(str_temp)


    pd_data = {"comment":str_youtube_comments, 'likes':str_like}
    youtube_pd = pd.DataFrame(pd_data)
    driver.close()

    return youtube_pd



In [ ]:
# 슬기로운 노다가생활

url_sulgi = """https://www.youtube.com/watch?v=0eHU3MUk6rc&list=PLdyB3s37qpTNCur1L88nYLJQgZfgzK5Sn&index=73
https://www.youtube.com/watch?v=ojkadJ0g3Cc&list=PLdyB3s37qpTNCur1L88nYLJQgZfgzK5Sn&index=67
https://www.youtube.com/watch?v=hu1dUmaB1ik&list=PLdyB3s37qpTNCur1L88nYLJQgZfgzK5Sn&index=60
https://www.youtube.com/watch?v=IyxpeEgV2sY&list=PLdyB3s37qpTNCur1L88nYLJQgZfgzK5Sn&index=57
https://www.youtube.com/watch?v=8VEgHf47CuM&list=PLdyB3s37qpTNCur1L88nYLJQgZfgzK5Sn&index=48
https://www.youtube.com/watch?v=TFODQ9aJQFs&list=PLdyB3s37qpTNCur1L88nYLJQgZfgzK5Sn&index=42
https://www.youtube.com/watch?v=JG-Yr2q-2VQ&list=PLdyB3s37qpTNCur1L88nYLJQgZfgzK5Sn&index=36
https://www.youtube.com/watch?v=2_l_4fgEhYQ&list=PLdyB3s37qpTNCur1L88nYLJQgZfgzK5Sn&index=30
https://www.youtube.com/watch?v=xiKULEAbTRw&list=PLdyB3s37qpTNCur1L88nYLJQgZfgzK5Sn&index=23
https://www.youtube.com/watch?v=zByR5OXaSsk&list=PLdyB3s37qpTNCur1L88nYLJQgZfgzK5Sn&index=17
https://www.youtube.com/watch?v=NwVdfT4E7dI&list=PLdyB3s37qpTNCur1L88nYLJQgZfgzK5Sn&index=10"""

url_sulgi = url_sulgi.split("\n")

In [ ]:
df_all = pd.DataFrame([], columns=['number','comment', 'likes'])

for i,url in enumerate(url_sulgi):
    df = get_comments_df(url)
    df['number'] = i+2
    df_all = df_all.append(df)
    print(i, df_all.shape)


df_all

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


0 (10, 3)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


1 (24, 3)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


9 (698, 3)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


10 (776, 3)


,number,comment,likes
0,2,0:13 0:13 내 수술 들어올래? 이거 진짜 미친 거 아님? 누가 수술 들어올래...,59
1,2,조정석 너무잘생김 ..,32
2,2,화신이가 수술들어오라는 것 같앜ㅋㅋㅋㅋ♡♡ 짜릿햌ㅋㅋㅋㅌㅋ,13
3,2,내 심장부터 먼저 수술해라...,13
4,2,미처ㅠㅠㅠㅠㅠㅠㅠㅠ이 영상보고 오나귀 다시 정주행하러 달려갑니다ㅜㅜㅜㅜㅜ하ㅜㅜ,8
...,...,...,...
73,12,English pleyazzzzz,0
74,12,아..... 송화 치홍에 미련을 못버렸었는데.... 오늘 폭망이네.... 송화 익준...,1
75,12,익송 익송 익송 익송 익송 익송 익송 익송 익송 익송 익송 익송 익송 익송 익송 익...,0
76,12,솔직히 채송화 반말하는 안치홍한테 설렜잖아ㅜㅜ 설렜다고 말해!! 왜 말을 못하냐고!!,3


In [ ]:
df_all.to_csv("슬기로운의사생활예고편댓글_1_12.csv",index=False)

# 스크립트

In [ ]:

def get_script_df(url):
    driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)
    driver.get(url)
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(2.0)       # 인터발 1이상으로 줘야 데이터 취득가능(롤링시 데이터 로딩 시간 때문)
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

        if new_page_height == last_page_height:
            break
        last_page_height = new_page_height
    html_source = driver.page_source
    

    # HTML 태크 크롤링 작업
    soup = BeautifulSoup(html_source, "html")
    print(soup.getText())
    youtube_user_scripts = soup.select("cue-group-start-offset style-scope ytd-transcript-body-renderer")
    print(youtube_user_scripts)
    str_youtube_scripts = []  # 스크립트

    # REPLACE DATA
    for i in range(len(youtube_user_scripts)):
        str_tmp = str(youtube_user_scripts[i].text)
        str_tmp = str_tmp.replace('\n', '')
        str_tmp = str_tmp.replace('\t', '')
        str_tmp = str_tmp.replace('   ','')
        str_youtube_scripts.append(str_tmp)


    pd_data = {"script":str_youtube_scripts}
    youtube_pd = pd.DataFrame(pd_data)
    driver.close()

    return youtube_pd



In [ ]:
get_script_df("https://www.youtube.com/watch?v=8VEgHf47CuM&list=PLdyB3s37qpTNCur1L88nYLJQgZfgzK5Sn&index=48")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until
